In [ ]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480

myColor = [[5,107,0,19,255,255],
          [133,56,0,159,156,255]
          ]

myColorValues =[[51,153,255],[255,0,255]]

myPoints = [] #[x,y,colorId]

In [ ]:
cap = cv2.VideoCapture(0)


In [ ]:
# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Unable to open camera.")
    exit()

cap.set(3, frameWidth)
cap.set(4, frameHeight)

In [ ]:
def findColor(img, myColors,myColorValues):
    count = 0
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    newPoints = []
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imgHSV, lower, upper)
        x,y = getContours(mask)
        cv2.circle(imgResult,(x,y),10,myColorValues[count],cv2.FILLED)
        if x!=0 and y !=0:
            newPoints.append([x,y,count])
        count+=1
    return newPoints

In [ ]:
def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h =0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(imgResult, [cnt], -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x, y, w, h = cv2.boundingRect(approx)
    return x+w//2,y

In [ ]:
def drawOnCanvas(myPoints,myColorValues):
    for point in myPoints:
        cv2.circle(imgResult,(point[0],point[1]),10,myColorValues[point[2]],cv2.FILLED)

In [ ]:
 #continuously captures frames from the camera and processes them
while True:

    success, img = cap.read()
    imgResult = img.copy()
    newPoints = findColor(img,myColor,myColorValues)
    if len(newPoints)!=0:
        for newP in newPoints:
            myPoints.append(newP)
    if len(newPoints)!=0:
        drawOnCanvas(myPoints,myColorValues)
    cv2.imshow("result",imgResult)
    if not success:
        print("Error: Failed to read frame from the camera.")
        break

    # Check if the captured frame is empty
    if img is None:
        print("Error: Empty frame captured from the camera.")
        continue


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:

# Release the camera properly
cap.release()
cv2.destroyAllWindows()